In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (accuracy_score, 
                             precision_score,
                             recall_score, 
                             f1_score)
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

In [8]:
data = pd.read_csv("../../data/normalized_tweets.csv")
data.head()

,tweet_text,cyberbullying_type
0,In words katandandre food crapilicious,not_cyberbullying
1,Why aussietv white,not_cyberbullying
2,classy whore Or red velvet cupcakes,not_cyberbullying
3,meh thanks heads concerned another angry dud...,not_cyberbullying
4,This ISIS account pretending Kurdish account ...,not_cyberbullying


In [3]:
#data = data[(data["cyberbullying_type"] != "other_cyberbullying")]

In [4]:
data['cyberbullying_type'].value_counts()

religion             7996
age                  7992
ethnicity            7958
gender               7906
not_cyberbullying    7816
Name: cyberbullying_type, dtype: int64

In [5]:
LE = LabelEncoder()
LE.fit(data['cyberbullying_type'])
data['cyberbullying_type'] = LE.transform(data['cyberbullying_type'])

In [6]:
x = data['tweet_text']
y = data['cyberbullying_type']

## Multinomial Naive Bayes Model

In [7]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
# Transform text data into TF-IDF features
X_tfidf = tfidf_vectorizer.fit_transform(x)

# Create Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Define K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store F1 scores and accuracy scores for each fold
f1_scores = []
accuracy_scores = []

# Perform K-fold cross-validation
for train_index, test_index in kfold.split(X_tfidf):
    X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Train classifier
    nb_classifier.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = nb_classifier.predict(X_test)
    
    # Calculate F1 score and accuracy
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    
    # Append scores to lists
    f1_scores.append(f1)
    accuracy_scores.append(accuracy)

# Calculate mean F1 score and accuracy across all folds
mean_f1_score = np.mean(f1_scores)
mean_accuracy = np.mean(accuracy_scores)

print("Mean F1 Score:", mean_f1_score)
print("Mean Accuracy:", mean_accuracy)

KeyError: '[23718, 23719, 23720, 23721, 23723, 23724, 23726, 23727, 23729, 23730, 23731, 23732, 23733, 23734, 23735, 23736, 23738, 23739, 23740, 23741, 23744, 23745, 23747, 23748, 23749, 23751, 23752, 23753, 23754, 23755, 23756, 23757, 23758, 23759, 23760, 23761, 23762, 23763, 23764, 23765, 23766, 23767, 23768, 23769, 23770, 23772, 23775, 23776, 23777, 23778, 23779, 23781, 23782, 23783, 23785, 23786, 23787, 23788, 23789, 23790, 23791, 23792, 23793, 23794, 23796, 23797, 23798, 23799, 23800, 23802, 23803, 23804, 23805, 23806, 23809, 23810, 23812, 23813, 23814, 23815, 23816, 23817, 23818, 23820, 23821, 23822, 23823, 23824, 23827, 23829, 23830, 23831, 23832, 23833, 23834, 23835, 23836, 23837, 23838, 23839, 23840, 23841, 23842, 23843, 23844, 23845, 23846, 23847, 23849, 23851, 23852, 23853, 23854, 23855, 23856, 23857, 23858, 23859, 23860, 23861, 23862, 23863, 23864, 23865, 23866, 23867, 23869, 23870, 23871, 23872, 23874, 23875, 23876, 23877, 23879, 23880, 23881, 23883, 23884, 23885, 23887, 2

## Binary Naive Bayes

### Religion vs Not_cyberbullying

In [23]:
from sklearn.naive_bayes import BernoulliNB

In [96]:
# religion = 5
# age = 0
# ethnicity = 1
# gender = 2
# not_cyberbullying = 3
# other_cyberbullying = 4
data['cyberbullying_type'].value_counts()

5    7996
0    7992
1    7958
2    7906
3    7816
4    7613
Name: cyberbullying_type, dtype: int64

In [97]:
# select religion and not_cyberbullyng
data2 = data[(data["cyberbullying_type"] == 5) | (data["cyberbullying_type"] == 3)]

In [98]:
data2['cyberbullying_type'].value_counts()

5    7996
3    7816
Name: cyberbullying_type, dtype: int64

In [99]:
data2

,tweet_text,cyberbullying_type
0,In words katandandre food crapilicious,3
1,Why aussietv white,3
2,classy whore Or red velvet cupcakes,3
3,meh thanks heads concerned another angry dud...,3
4,This ISIS account pretending Kurdish account ...,3
...,...,...
23713,Can imagine Christians came together like 5 ti...,5
23714,So support justice initial problem It morphed...,5
23715,If harbour doubts Muslims believe sharia note...,5
23716,One thing Muslims want exterminate everyone M...,5


In [100]:
# Separate features and target
X = data2['tweet_text']
y = data2['cyberbullying_type']

# Map labels to binary: 1 for cyberbullying, 0 for non-cyberbullying
y_binary = (y == 5).astype(int)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform text data into TF-IDF features
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binary, test_size=0.2, random_state=42)

# Create Bernoulli Naive Bayes classifier
nb_classifier = BernoulliNB()

# Train classifier
nb_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = nb_classifier.predict(X_test)

# Calculate F1 score and accuracy
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("F1 Score:", f1)
print("Accuracy:", accuracy)


F1 Score: 0.94336
Accuracy: 0.9440404679102118


### Gender vs not cyberbullying

In [101]:
data3 = data[(data["cyberbullying_type"] == 2) | (data["cyberbullying_type"] == 3)]

data3['cyberbullying_type'].value_counts()

# Separate features and target
X = data3['tweet_text']
y = data3['cyberbullying_type']

# Map labels to binary: 1 for cyberbullying, 0 for non-cyberbullying
y_binary = (y == 2).astype(int)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform text data into TF-IDF features
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binary, test_size=0.2, random_state=42)

# Create Bernoulli Naive Bayes classifier
nb_classifier = BernoulliNB()

# Train classifier
nb_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = nb_classifier.predict(X_test)

# Calculate F1 score and accuracy
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("F1 Score:", f1)
print("Accuracy:", accuracy)


F1 Score: 0.8657807308970098
Accuracy: 0.8715421303656598


### Ethnicity vs not cyberbullying

In [104]:
data4 = data[(data["cyberbullying_type"] == 1) | (data["cyberbullying_type"] == 3)]

data4['cyberbullying_type'].value_counts()

# Separate features and target
X = data4['tweet_text']
y = data4['cyberbullying_type']

# Map labels to binary: 1 for cyberbullying, 0 for non-cyberbullying
y_binary = (y == 1).astype(int)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform text data into TF-IDF features
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binary, test_size=0.2, random_state=42)

# Create Bernoulli Naive Bayes classifier
nb_classifier = BernoulliNB()

# Train classifier
nb_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = nb_classifier.predict(X_test)

# Calculate F1 score and accuracy
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("F1 Score:", f1)
print("Accuracy:", accuracy)


F1 Score: 0.9710144927536233
Accuracy: 0.9708399366085578


### Age vs not cyberbullying

In [105]:
data5 = data[(data["cyberbullying_type"] == 0) | (data["cyberbullying_type"] == 3)]

data5['cyberbullying_type'].value_counts()

# Separate features and target
X = data5['tweet_text']
y = data5['cyberbullying_type']

# Map labels to binary: 1 for cyberbullying, 0 for non-cyberbullying
y_binary = (y == 0).astype(int)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform text data into TF-IDF features
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binary, test_size=0.2, random_state=42)

# Create Bernoulli Naive Bayes classifier
nb_classifier = BernoulliNB()

# Train classifier
nb_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = nb_classifier.predict(X_test)

# Calculate F1 score and accuracy
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("F1 Score:", f1)
print("Accuracy:", accuracy)


F1 Score: 0.9631920816579029
Accuracy: 0.9623655913978495


## Combined binary (tweets from each category)

In [73]:
# merge tweets (1953 for each category)
# Numero di tweet per categoria (tranne 'other cyberbullying')
n = 1953

# Leggi il dataset
df = pd.read_csv('../../data/normalized_tweets.csv')
df = df[df['cyberbullying_type'] != 'other_cyberbullying']
#df = df[df['cyberbullying_type'] != 'gender'] # how many duplicated tweets ?

# Crea un DataFrame vuoto per i risultati
result_df = pd.DataFrame(columns=['tweet_text', 'label'])

# Estraiamo i tweet per ogni categoria
for category in df['cyberbullying_type'].unique():
    if category != 'not_cyberbullying':
        sampled_df = df[df['cyberbullying_type'] == category].sample(n=n, replace=False)
        sampled_df.loc[:, 'label'] = 0  
        result_df = pd.concat([result_df, sampled_df[['tweet_text', 'label']]], ignore_index=True)
    else:
        other_cyberbullying_df = df[df['cyberbullying_type'] == category].copy() 
        other_cyberbullying_df.loc[:, 'label'] = 1 
        result_df = pd.concat([result_df, other_cyberbullying_df[['tweet_text', 'label']]], ignore_index=True)

In [71]:
result_df

,tweet_text,label
0,In words katandandre food crapilicious,1
1,Why aussietv white,1
2,classy whore Or red velvet cupcakes,1
3,meh thanks heads concerned another angry dud...,1
4,This ISIS account pretending Kurdish account ...,1
...,...,...
17811,YOU mad bitch RT FUCK OBAMA dumb ass nigger,0
17812,stfu dumb cunt call fucking nigger think big s...,0
17813,Lmao madt finish Black Salesman Colored Prod...,0
17814,The difference American Revolution revolution ...,0


In [74]:
result_df["label"].value_counts()

1    7816
0    7812
Name: label, dtype: int64

In [75]:
# Separate features and target
X = result_df['tweet_text']
y = result_df['label']

# map to int type
y=y.astype('int')

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform text data into TF-IDF features
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2)

# Create Bernoulli Naive Bayes classifier
nb_classifier = BernoulliNB()

# Train classifier
nb_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = nb_classifier.predict(X_test)

# Calculate F1 score and accuracy
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("F1 Score:", f1)
print("Accuracy:", accuracy)

F1 Score: 0.9055334760012229
Accuracy: 0.9011516314779271


## Grid search

In [79]:
from sklearn.model_selection import GridSearchCV

# Load example dataset (you can replace this with your dataset)
data = pd.read_csv('../../data/normalized_tweets.csv')
data = data[data['cyberbullying_type'] != 'other_cyberbullying']

# Define a pipeline with CountVectorizer and MultinomialNB
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Define the parameters grid for grid search
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (1, 2, 3),
    'vect__ngram_range': ((1, 1), (1, 2)),  
    'clf__alpha': (0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.tweet_text, data.cyberbullying_type)

# Print best parameters and best score
print("Best parameters found:")
print(grid_search.best_params_)
print("Best score found:")
print(grid_search.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


KeyboardInterrupt: 